In [1]:
using JuMP
using SCIP
using Statistics
using MathOptInterface
using Plots
gr()

Plots.GRBackend()

インスタンスのうち計算時間が大きそうなものを探してくる

see: https://zenn.dev/kounoike/articles/20210327-hard-knapsack#pisinger%E3%81%AE%E3%82%B8%E3%82%A7%E3%83%8D%E3%83%AC%E3%83%BC%E3%82%BF

In [2]:
# 関数
function solve(W, V, Capacity; ts=1, level=1, gap=0.01)
    N = length(W)
    
    # SCIPのみ
    model = Model(SCIP.Optimizer)
    set_optimizer_attribute(model, "display/verblevel", level)
    set_optimizer_attribute(model, "limits/time", ts)
    set_optimizer_attribute(model, "limits/gap", gap)
    
    @variable(model, x[1:N], Bin)
    @objective(model, Max, sum(V[i] * x[i] for i in 1:N))
    @constraint(model, sum(W[i] * x[i] for i in 1:N) <= Capacity)
    optimize!(model)
    
    if MOI.get(model, MOI.TerminationStatus()) == MOI.OPTIMAL # check
        obj = objective_value(model)
        resx = value.(x)
        sol = [i for i in 1:N if resx[i] > 0.5]
        return obj, sol
    else
        return nothing, nothing
    end
end

solve (generic function with 1 method)

In [3]:
# 1回動かしておくと良い
W = [4, 2, 2, 1, 10]
V = [12, 2, 1, 1, 4]
Capacity = 15
obj, sol = solve(W, V, Capacity; level=0)
println(obj)
println(sol)

17.0
[1, 4, 5]


In [6]:
op = open("result.csv", "w")

for filename in sort(readdir("download_instances/"))
    !(endswith(filename, ".csv")) && continue
    
    n = parse(Int, split(filename, "_")[3])
        
    open("./download_instances/$(filename)", "r") do fp
        lines = readlines(fp)
        
        indices = []
        for idx in 1:length(lines)
            if startswith(lines[idx], "knap")
                push!(indices, idx)
            end
        end
    
        
        res_ratio = []
        res_time = []
        for s in indices
            # n = parse(Int, split(lines[s + 1], " ")[2])
            c = parse(Int, split(lines[s + 2], " ")[2])
            z = parse(Int, split(lines[s + 3], " ")[2])
            t = parse(Float64, split(lines[s + 4], " ")[2])
            
            (t < 0.5) && continue
            
            W = []
            V = []
            data_sol = []
            w = 0
            
            for line in lines[s+5:s+5+n-1]
                (line[1] == "-") && continue
                num, value, weight, sol = parse.(Int, split(line, ","))
                push!(W, weight)
                push!(V, value)
                if sol > 0.5
                    w += weight
                    push!(data_sol, num)
                end                
            end

            # Solve by SCIP
            ts = time()
            obj, sol = solve(W, V, c; level=0, ts=30, gap=0.001)
            t2 = time() - ts
            push!(res_ratio, obj / z)
            push!(res_time, t2)
        end
        
        if !isempty(res_ratio)
            println(filename, ",", mean(res_ratio), ",", mean(res_time))
            println(op, filename, ",", mean(res_ratio), ",", mean(res_time))
        end
    end
end

close(op)

knapPI_11_10000_1000.csv,0.9998233048105855,0.9160730242729187
knapPI_12_10000_1000.csv,0.9998998854346434,0.9064184308052063
knapPI_12_5000_1000.csv,0.9998473151115029,0.5555031299591064
knapPI_13_10000_1000.csv,0.9997428012813293,0.8743130874633789
knapPI_13_5000_1000.csv,0.9997680048804578,0.5152662595113119
knapPI_15_10000_1000.csv,0.9996628777251206,0.7030484318733216
knapPI_15_5000_1000.csv,0.9996096930304744,0.4814458063670567
